In [1]:
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from main.layout import Layout
from main.algorithms import enum_layout, enum_layout_wo_rdt, init_S, coarse_to_fined
from main.models import BasicBlock, ResidualBlock
from main.head import ASPPHeadNode
from main.trainer import Trainer
from data.nyuv2_dataloader_adashare import NYU_v2
from data.pixel2pixel_loss import NYUCriterions
from data.pixel2pixel_metrics import NYUMetrics

In [2]:
class LayoutBlock(nn.Module):
    def __init__(self, residual_block, task_set, layer_idx):
        super(LayoutBlock, self).__init__() 
        self.residual_block = residual_block
        self.task_set = task_set
        self.layer_idx = layer_idx
        
        # set outside 
        self.parent_block = [] 
        self.output = None
        
    def forward(self, x=None):
        if len(self.parent_block) == 0:
            if x is not None:
                return self.residual_block(x)
            else:
                print('Wrong Input Data!')
                return
        else:
            return self.residual_block(self.parent_block[0].output)

In [3]:
affine_par = True
class Deeplab_ResNet_Backbone_Layout(nn.Module):
    def __init__(self, block, layers, layout):
        super(Deeplab_ResNet_Backbone_Layout, self).__init__()
        
        # Step 1: Sanity Check - The number of blocks in Deeplab_Resnet34 should be 17
        if sum(layers) + 1 != layout.B:
            print('Given layout cannot construct multi-task model of Deeplab_Resnet34 because of the incompatiable number of blocks.')
            return
        
        # Step 2: Initiate Backbone Blocks - Modules in self.blocks are not used in forward
        self.inplanes = 64
        self.layout = layout
    
        strides = [1, 2, 1, 1]
        dilations = [1, 1, 2, 4]
        filt_sizes = [64, 128, 256, 512]
        self.blocks = []
        self.layer_config = layers
        
        branch_cnt = 0
        seed = self._make_seed()
        self.__add_to_blocks(seed)
        branch_cnt += 1
        
        for segment, num_blocks in enumerate(self.layer_config):
            filt_size, num_blocks, stride, dilation = filt_sizes[segment],layers[segment],strides[segment],dilations[segment]
            for b_idx in range(num_blocks):
                blocklayer = self._make_blocklayer(b_idx, block, filt_size, stride=stride, dilation=dilation)
                self.__add_to_blocks(blocklayer)
#         self.blocks = nn.ModuleList(self.blocks)
        
        # Step 3: Copy Blocks to Construct Multi-Task Model
        self.mtl_blocks = []
        for layer_idx in range(self.layout.B):
            basic_block = self.blocks[layer_idx]
            for task_set in self.layout.state[layer_idx]:
                layoutblock = LayoutBlock(copy.deepcopy(basic_block), task_set, layer_idx)
                # set parent block except the first layer
                if layer_idx != 0:
                    for block in self.mtl_blocks:
                        if block.layer_idx == layer_idx - 1 and task_set.issubset(block.task_set):
                            layoutblock.parent_block.append(block)
                            break
                self.mtl_blocks.append(layoutblock)
        self.mtl_blocks = nn.ModuleList(self.mtl_blocks)
        
        # Step 4: Initiate Weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, 0.01)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    # Functions for Backbone Blocks
    def _make_seed(self):
        seed = nn.Sequential(nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
                             nn.BatchNorm2d(64, affine=affine_par),
                             nn.ReLU(inplace=True),
                             nn.MaxPool2d(kernel_size=3, stride=2, padding=1, ceil_mode=True)) 
        return seed
    
    def _make_downsample(self, block, inplanes, planes, stride=1, dilation=1):
        downsample = None
        if stride != 1 or inplanes != planes * block.expansion or dilation == 2 or dilation == 4:
            downsample = nn.Sequential(nn.Conv2d(inplanes, planes * block.expansion, kernel_size=1, stride=stride, bias=False),
                                       nn.BatchNorm2d(planes * block.expansion, affine = affine_par))
        return downsample
    
    def _make_blocklayer(self, block_idx, block, planes, stride=1, dilation=1):
        ds = None
        if block_idx == 0:
            basic_block = block(self.inplanes, planes, stride, dilation=dilation)
            ds = self._make_downsample(block, self.inplanes, planes, stride=stride, dilation=dilation)
            self.inplanes = planes * block.expansion
        else:
            basic_block = block(self.inplanes, planes, dilation=dilation)
            
        blocklayer = ResidualBlock(basic_block, ds)
        return blocklayer
    
    def __add_to_blocks(self, block):
        self.blocks.append(block)
        return   
    
    def forward(self, x): 
        features = [0] * self.layout.T
        for block in self.mtl_blocks:
            block.output = block(x)
            if block.layer_idx == self.layout.B - 1:
                for task in block.task_set:
                    features[task] = block.output
        return features

In [4]:
class Deeplab_ASPP_Layout(nn.Module):
    def __init__(self, layout, cls_num):
        super(Deeplab_ASPP_Layout, self).__init__()
        self.layout = layout
        self.backbone = Deeplab_ResNet_Backbone_Layout(BasicBlock, [3, 4, 6, 3], layout)
        self.heads = nn.ModuleDict()
        for task in cls_num:
            self.heads[task] = ASPPHeadNode(512, cls_num[task])
        
    def forward(self, x):
        features = self.backbone(x)
        output = {}
        idx = 0
        for task in self.heads:
            output[task] = self.heads[task](features[idx])
            idx += 1
        return output

In [5]:
# Enum all layouts for given T (3 for NYUv2) and B (17 for Deeplab_Resnet34, 5 for coarse version)
T = 3 
B = 5 

layout_list = [] 
S0 = init_S(T, B) # initial state
L = Layout(T, B, S0) # initial layout
layout_list.append(L)

enum_layout_wo_rdt(L, layout_list)

In [6]:
layout_list[30]

[[{0, 1, 2}], [{0, 1, 2}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}]]

In [4]:
layout_list[251]

[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}]]

In [5]:
layout_list[278]

[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {1}, {0}], [{2}, {1}, {0}]]

In [6]:
# Transform layout to multi-task model backbone
layout = layout_list[2]
backbone = Deeplab_ResNet_Backbone_Layout(BasicBlock, [3, 4, 6, 3], layout)
print(layout)

[[{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}]]


In [8]:
x = torch.rand(1,3,224,224)
y = backbone(x)

In [7]:
print(backbone)

Deeplab_ResNet_Backbone_Layout(
  (mtl_blocks): ModuleList(
    (0): LayoutBlock(
      (residual_block): Sequential(
        (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=True)
      )
    )
    (1): LayoutBlock(
      (residual_block): Sequential(
        (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=True)
      )
    )
    (2): LayoutBlock(
      (residual_block): Sequential(
        (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, mome

# Multi-Task Model from Layout

In [7]:
dataroot = '/mnt/nfs/work1/huiguan/lijunzhang/policymtl/data/NYUv2/'
tasks = ['segment_semantic','normal','depth_zbuffer']
task_cls_num = {'segment_semantic': 40, 'normal':3, 'depth_zbuffer': 1}

dataset = NYU_v2(dataroot, 'train', crop_h=321, crop_w=321)
trainDataloader = DataLoader(dataset, 16, shuffle=True)

dataset = NYU_v2(dataroot, 'test', crop_h=321, crop_w=321)
valDataloader = DataLoader(dataset, 16, shuffle=True)

criterionDict = {}
metricDict = {}
for task in tasks:
    criterionDict[task] = NYUCriterions(task)
    metricDict[task] = NYUMetrics(task)

In [8]:
layout = layout_list[30]

# if coarse version
fined_B = 17
mapping = {0:[0], 1:[1,2,3], 2:[4,5,6,7], 3:[8,9,10,11,12,13], 4:[14,15,16], 5:[17]}
layout = coarse_to_fined(layout, fined_B, mapping)

In [ ]:
model = Deeplab_ASPP_Layout(layout, task_cls_num)
model = model.cuda()

In [8]:
loss_lambda = {'segment_semantic': 1, 'normal':1, 'depth_zbuffer': 1}
checkpoint = '/mnt/nfs/work1/huiguan/lijunzhang/multibranch/checkpoint/NYUv2/exp/'

trainer = Trainer(model, tasks, trainDataloader, valDataloader, criterionDict, metricDict)
# trainer.train_step(loss_lambda)
trainer.train(20000, loss_lambda, checkpoint)

[Iter 50 Task segm] Train Loss: 3.4201
[Iter 50 Task norm] Train Loss: 0.0862
[Iter 50 Task dept] Train Loss: 2.3696
[Iter 50 Total] Train Loss: 5.8759
[Iter 100 Task segm] Train Loss: 2.8085
[Iter 100 Task norm] Train Loss: 0.0678
[Iter 100 Task dept] Train Loss: 1.1164
[Iter 100 Total] Train Loss: 3.9927
[Iter 150 Task segm] Train Loss: 2.6478
[Iter 150 Task norm] Train Loss: 0.0668
[Iter 150 Task dept] Train Loss: 1.1025
[Iter 150 Total] Train Loss: 3.8171
[Iter 200 Task segm] Train Loss: 2.6405
[Iter 200 Task norm] Train Loss: 0.0688
[Iter 200 Task dept] Train Loss: 1.0715
[Iter 200 Total] Train Loss: 3.7808
[Iter 200 Task segm] Val Loss: 2.5774
{'mIoU': 0.1362, 'Pixel Acc': 0.3458}
[Iter 200 Task norm] Val Loss: 0.0675
{'Angle Mean': 18.8468, 'Angle Median': 16.875, 'Angle 11.25': 22.3407, 'Angle 22.5': 71.9892, 'Angle 30': 87.5007}
[Iter 200 Task dept] Val Loss: 1.1259
{'abs_err': 1.1091, 'rel_err': 0.3575, 'sigma_1.25': 26.8402, 'sigma_1.25^2': 55.1159, 'sigma_1.25^3': 79.3055}


[Iter 1250 Task norm] Train Loss: 0.0667
[Iter 1250 Task dept] Train Loss: 1.0031
[Iter 1250 Total] Train Loss: 3.2363
[Iter 1300 Task segm] Train Loss: 2.1416
[Iter 1300 Task norm] Train Loss: 0.0668
[Iter 1300 Task dept] Train Loss: 0.9946
[Iter 1300 Total] Train Loss: 3.2030
[Iter 1350 Task segm] Train Loss: 2.0860
[Iter 1350 Task norm] Train Loss: 0.0677
[Iter 1350 Task dept] Train Loss: 0.9939
[Iter 1350 Total] Train Loss: 3.1476
[Iter 1400 Task segm] Train Loss: 2.1003
[Iter 1400 Task norm] Train Loss: 0.0677
[Iter 1400 Task dept] Train Loss: 1.0071
[Iter 1400 Total] Train Loss: 3.1751
[Iter 1400 Task segm] Val Loss: 2.1907
{'mIoU': 0.1082, 'Pixel Acc': 0.4007}
[Iter 1400 Task norm] Val Loss: 0.0687
{'Angle Mean': 19.1579, 'Angle Median': 17.5427, 'Angle 11.25': 21.0161, 'Angle 22.5': 70.4541, 'Angle 30': 87.7415}
[Iter 1400 Task dept] Val Loss: 0.8946
{'abs_err': 0.8857, 'rel_err': 0.3352, 'sigma_1.25': 41.4276, 'sigma_1.25^2': 72.2909, 'sigma_1.25^3': 88.5307}
[Iter 1450 Task s

[Iter 2450 Task segm] Train Loss: 1.7893
[Iter 2450 Task norm] Train Loss: 0.0674
[Iter 2450 Task dept] Train Loss: 0.9274
[Iter 2450 Total] Train Loss: 2.7840
[Iter 2500 Task segm] Train Loss: 1.7881
[Iter 2500 Task norm] Train Loss: 0.0663
[Iter 2500 Task dept] Train Loss: 0.9289
[Iter 2500 Total] Train Loss: 2.7833
[Iter 2550 Task segm] Train Loss: 1.7536
[Iter 2550 Task norm] Train Loss: 0.0662
[Iter 2550 Task dept] Train Loss: 0.9206
[Iter 2550 Total] Train Loss: 2.7404
[Iter 2600 Task segm] Train Loss: 1.7630
[Iter 2600 Task norm] Train Loss: 0.0657
[Iter 2600 Task dept] Train Loss: 0.9218
[Iter 2600 Total] Train Loss: 2.7505
[Iter 2600 Task segm] Val Loss: 1.9770
{'mIoU': 0.1283, 'Pixel Acc': 0.4405}
[Iter 2600 Task norm] Val Loss: 0.0678
{'Angle Mean': 18.4531, 'Angle Median': 16.2714, 'Angle 11.25': 27.3072, 'Angle 22.5': 71.3806, 'Angle 30': 86.5461}
[Iter 2600 Task dept] Val Loss: 0.8398
{'abs_err': 0.8326, 'rel_err': 0.3316, 'sigma_1.25': 45.3752, 'sigma_1.25^2': 74.7575, '

[Iter 3650 Task segm] Train Loss: 1.5326
[Iter 3650 Task norm] Train Loss: 0.0658
[Iter 3650 Task dept] Train Loss: 0.8993
[Iter 3650 Total] Train Loss: 2.4978
[Iter 3700 Task segm] Train Loss: 1.4690
[Iter 3700 Task norm] Train Loss: 0.0646
[Iter 3700 Task dept] Train Loss: 0.8960
[Iter 3700 Total] Train Loss: 2.4296
[Iter 3750 Task segm] Train Loss: 1.5093
[Iter 3750 Task norm] Train Loss: 0.0654
[Iter 3750 Task dept] Train Loss: 0.8992
[Iter 3750 Total] Train Loss: 2.4739
[Iter 3800 Task segm] Train Loss: 1.4682
[Iter 3800 Task norm] Train Loss: 0.0646
[Iter 3800 Task dept] Train Loss: 0.9200
[Iter 3800 Total] Train Loss: 2.4529
[Iter 3800 Task segm] Val Loss: 1.7696
{'mIoU': 0.17, 'Pixel Acc': 0.4965}
[Iter 3800 Task norm] Val Loss: 0.0674
{'Angle Mean': 18.4532, 'Angle Median': 16.2881, 'Angle 11.25': 25.2875, 'Angle 22.5': 73.4055, 'Angle 30': 86.6144}
[Iter 3800 Task dept] Val Loss: 0.8375
{'abs_err': 0.8284, 'rel_err': 0.3135, 'sigma_1.25': 44.0815, 'sigma_1.25^2': 75.3644, 'si

[Iter 4850 Task segm] Train Loss: 1.1772
[Iter 4850 Task norm] Train Loss: 0.0646
[Iter 4850 Task dept] Train Loss: 0.8330
[Iter 4850 Total] Train Loss: 2.0749
[Iter 4900 Task segm] Train Loss: 1.1624
[Iter 4900 Task norm] Train Loss: 0.0639
[Iter 4900 Task dept] Train Loss: 0.8520
[Iter 4900 Total] Train Loss: 2.0783
[Iter 4950 Task segm] Train Loss: 1.1396
[Iter 4950 Task norm] Train Loss: 0.0641
[Iter 4950 Task dept] Train Loss: 0.8302
[Iter 4950 Total] Train Loss: 2.0338
[Iter 5000 Task segm] Train Loss: 1.1720
[Iter 5000 Task norm] Train Loss: 0.0637
[Iter 5000 Task dept] Train Loss: 0.8355
[Iter 5000 Total] Train Loss: 2.0712
[Iter 5000 Task segm] Val Loss: 1.5857
{'mIoU': 0.2211, 'Pixel Acc': 0.5366}
[Iter 5000 Task norm] Val Loss: 0.0665
{'Angle Mean': 18.342, 'Angle Median': 16.4755, 'Angle 11.25': 26.1483, 'Angle 22.5': 73.1232, 'Angle 30': 86.8553}
[Iter 5000 Task dept] Val Loss: 0.8142
{'abs_err': 0.8048, 'rel_err': 0.2946, 'sigma_1.25': 45.4561, 'sigma_1.25^2': 76.5135, 's

[Iter 6050 Task segm] Train Loss: 1.0181
[Iter 6050 Task norm] Train Loss: 0.0629
[Iter 6050 Task dept] Train Loss: 0.8186
[Iter 6050 Total] Train Loss: 1.8996
[Iter 6100 Task segm] Train Loss: 0.9709
[Iter 6100 Task norm] Train Loss: 0.0632
[Iter 6100 Task dept] Train Loss: 0.7942
[Iter 6100 Total] Train Loss: 1.8283
[Iter 6150 Task segm] Train Loss: 1.0354
[Iter 6150 Task norm] Train Loss: 0.0632
[Iter 6150 Task dept] Train Loss: 0.7925
[Iter 6150 Total] Train Loss: 1.8911
[Iter 6200 Task segm] Train Loss: 1.0305
[Iter 6200 Task norm] Train Loss: 0.0635
[Iter 6200 Task dept] Train Loss: 0.7934
[Iter 6200 Total] Train Loss: 1.8874
[Iter 6200 Task segm] Val Loss: 1.6042
{'mIoU': 0.2314, 'Pixel Acc': 0.535}
[Iter 6200 Task norm] Val Loss: 0.0662
{'Angle Mean': 18.2039, 'Angle Median': 16.569, 'Angle 11.25': 25.2205, 'Angle 22.5': 73.1673, 'Angle 30': 86.529}
[Iter 6200 Task dept] Val Loss: 0.7613
{'abs_err': 0.7589, 'rel_err': 0.3166, 'sigma_1.25': 49.8248, 'sigma_1.25^2': 78.6801, 'sig

KeyboardInterrupt: 

In [8]:
# reload
loss_lambda = {'segment_semantic': 1, 'normal':1, 'depth_zbuffer': 1}
checkpoint = '/mnt/nfs/work1/huiguan/lijunzhang/multibranch/checkpoint/NYUv2/exp/'

trainer = Trainer(model, tasks, trainDataloader, valDataloader, criterionDict, metricDict)
# trainer.train_step(loss_lambda)
trainer.train(20000, loss_lambda, checkpoint, reload='segment_semantic_normal_depth_zbuffer.model')

[Iter 6650 Task segm] Train Loss: 0.9414
[Iter 6650 Task norm] Train Loss: 0.0638
[Iter 6650 Task dept] Train Loss: 0.7629
[Iter 6650 Total] Train Loss: 1.7680
[Iter 6700 Task segm] Train Loss: 0.9489
[Iter 6700 Task norm] Train Loss: 0.0636
[Iter 6700 Task dept] Train Loss: 0.8047
[Iter 6700 Total] Train Loss: 1.8172
[Iter 6750 Task segm] Train Loss: 0.9688
[Iter 6750 Task norm] Train Loss: 0.0641
[Iter 6750 Task dept] Train Loss: 0.7942
[Iter 6750 Total] Train Loss: 1.8271
[Iter 6800 Task segm] Train Loss: 0.9297
[Iter 6800 Task norm] Train Loss: 0.0643
[Iter 6800 Task dept] Train Loss: 0.7946
[Iter 6800 Total] Train Loss: 1.7886
[Iter 6800 Task segm] Val Loss: 1.5367
{'mIoU': 0.2362, 'Pixel Acc': 0.5506}
[Iter 6800 Task norm] Val Loss: 0.0673
{'Angle Mean': 18.0348, 'Angle Median': 16.4972, 'Angle 11.25': 29.9483, 'Angle 22.5': 70.57, 'Angle 30': 85.9449}
[Iter 6800 Task dept] Val Loss: 0.8264
{'abs_err': 0.8186, 'rel_err': 0.2991, 'sigma_1.25': 44.9867, 'sigma_1.25^2': 76.1334, 'si

[Iter 7850 Task segm] Train Loss: 0.8556
[Iter 7850 Task norm] Train Loss: 0.0641
[Iter 7850 Task dept] Train Loss: 0.7401
[Iter 7850 Total] Train Loss: 1.6598
[Iter 7900 Task segm] Train Loss: 0.8820
[Iter 7900 Task norm] Train Loss: 0.0618
[Iter 7900 Task dept] Train Loss: 0.7450
[Iter 7900 Total] Train Loss: 1.6887
[Iter 7950 Task segm] Train Loss: 0.8416
[Iter 7950 Task norm] Train Loss: 0.0622
[Iter 7950 Task dept] Train Loss: 0.7327
[Iter 7950 Total] Train Loss: 1.6365
[Iter 8000 Task segm] Train Loss: 0.8507
[Iter 8000 Task norm] Train Loss: 0.0639
[Iter 8000 Task dept] Train Loss: 0.7484
[Iter 8000 Total] Train Loss: 1.6630
[Iter 8000 Task segm] Val Loss: 1.5086
{'mIoU': 0.2496, 'Pixel Acc': 0.5619}
[Iter 8000 Task norm] Val Loss: 0.0660
{'Angle Mean': 18.0565, 'Angle Median': 16.3573, 'Angle 11.25': 28.5937, 'Angle 22.5': 72.7581, 'Angle 30': 86.6814}
[Iter 8000 Task dept] Val Loss: 0.7248
{'abs_err': 0.724, 'rel_err': 0.3169, 'sigma_1.25': 52.0354, 'sigma_1.25^2': 80.0685, 's

[Iter 9050 Task segm] Train Loss: 0.6717
[Iter 9050 Task norm] Train Loss: 0.0625
[Iter 9050 Task dept] Train Loss: 0.6876
[Iter 9050 Total] Train Loss: 1.4218
[Iter 9100 Task segm] Train Loss: 0.6528
[Iter 9100 Task norm] Train Loss: 0.0620
[Iter 9100 Task dept] Train Loss: 0.6726
[Iter 9100 Total] Train Loss: 1.3875
[Iter 9150 Task segm] Train Loss: 0.6609
[Iter 9150 Task norm] Train Loss: 0.0626
[Iter 9150 Task dept] Train Loss: 0.6680
[Iter 9150 Total] Train Loss: 1.3915
[Iter 9200 Task segm] Train Loss: 0.6408
[Iter 9200 Task norm] Train Loss: 0.0615
[Iter 9200 Task dept] Train Loss: 0.6653
[Iter 9200 Total] Train Loss: 1.3676
[Iter 9200 Task segm] Val Loss: 1.4979
{'mIoU': 0.2609, 'Pixel Acc': 0.5699}
[Iter 9200 Task norm] Val Loss: 0.0651
{'Angle Mean': 17.9139, 'Angle Median': 16.2789, 'Angle 11.25': 29.1505, 'Angle 22.5': 72.5048, 'Angle 30': 86.8548}
[Iter 9200 Task dept] Val Loss: 0.6814
{'abs_err': 0.6777, 'rel_err': 0.2749, 'sigma_1.25': 55.1871, 'sigma_1.25^2': 82.9686, '

[Iter 10250 Task segm] Train Loss: 0.6146
[Iter 10250 Task norm] Train Loss: 0.0612
[Iter 10250 Task dept] Train Loss: 0.6331
[Iter 10250 Total] Train Loss: 1.3089
[Iter 10300 Task segm] Train Loss: 0.6023
[Iter 10300 Task norm] Train Loss: 0.0628
[Iter 10300 Task dept] Train Loss: 0.6233
[Iter 10300 Total] Train Loss: 1.2884
[Iter 10350 Task segm] Train Loss: 0.5774
[Iter 10350 Task norm] Train Loss: 0.0627
[Iter 10350 Task dept] Train Loss: 0.6148
[Iter 10350 Total] Train Loss: 1.2549
[Iter 10400 Task segm] Train Loss: 0.5980
[Iter 10400 Task norm] Train Loss: 0.0601
[Iter 10400 Task dept] Train Loss: 0.6393
[Iter 10400 Total] Train Loss: 1.2975
[Iter 10400 Task segm] Val Loss: 1.5704
{'mIoU': 0.241, 'Pixel Acc': 0.5631}
[Iter 10400 Task norm] Val Loss: 0.0643
{'Angle Mean': 17.9832, 'Angle Median': 16.1788, 'Angle 11.25': 27.2915, 'Angle 22.5': 74.4564, 'Angle 30': 87.1801}
[Iter 10400 Task dept] Val Loss: 0.6455
{'abs_err': 0.6432, 'rel_err': 0.2583, 'sigma_1.25': 56.3355, 'sigma_1

KeyboardInterrupt: 

In [13]:
for name, param in model.named_parameters():
    print(name)

backbone.mtl_blocks.0.residual_block.0.weight
backbone.mtl_blocks.0.residual_block.1.weight
backbone.mtl_blocks.0.residual_block.1.bias
backbone.mtl_blocks.1.residual_block.0.weight
backbone.mtl_blocks.1.residual_block.1.weight
backbone.mtl_blocks.1.residual_block.1.bias
backbone.mtl_blocks.2.residual_block.0.weight
backbone.mtl_blocks.2.residual_block.1.weight
backbone.mtl_blocks.2.residual_block.1.bias
backbone.mtl_blocks.3.residual_block.block.conv1.weight
backbone.mtl_blocks.3.residual_block.block.bn1.weight
backbone.mtl_blocks.3.residual_block.block.bn1.bias
backbone.mtl_blocks.3.residual_block.block.conv2.weight
backbone.mtl_blocks.3.residual_block.block.bn2.weight
backbone.mtl_blocks.3.residual_block.block.bn2.bias
backbone.mtl_blocks.4.residual_block.block.conv1.weight
backbone.mtl_blocks.4.residual_block.block.bn1.weight
backbone.mtl_blocks.4.residual_block.block.bn1.bias
backbone.mtl_blocks.4.residual_block.block.conv2.weight
backbone.mtl_blocks.4.residual_block.block.bn2.wei

In [123]:
model.backbone.mtl_blocks[0].residual_block[0].weight

Parameter containing:
tensor([[[[-0.0760, -0.0185, -0.0045,  ..., -0.0666,  0.0398, -0.0463],
          [ 0.0192,  0.0383, -0.0655,  ..., -0.0483,  0.0584,  0.0642],
          [ 0.0607, -0.0671,  0.0245,  ...,  0.0189,  0.0043, -0.0241],
          ...,
          [ 0.0353,  0.0788,  0.0675,  ...,  0.0649, -0.0503, -0.0258],
          [ 0.0389, -0.0678,  0.0187,  ..., -0.0410, -0.0471, -0.0021],
          [-0.0303,  0.0325,  0.0399,  ...,  0.0383, -0.0728, -0.0163]],

         [[ 0.0251, -0.0353, -0.0466,  ...,  0.0166, -0.0488,  0.0640],
          [ 0.0646, -0.0685, -0.0664,  ...,  0.0109,  0.0040, -0.0491],
          [-0.0162,  0.0353,  0.0291,  ..., -0.0658,  0.0065,  0.0196],
          ...,
          [ 0.0184, -0.0147, -0.0767,  ..., -0.0552, -0.0178, -0.0662],
          [-0.0633,  0.0209,  0.0740,  ...,  0.0312, -0.0605, -0.0290],
          [-0.0342, -0.0763,  0.0609,  ..., -0.0716,  0.0591, -0.0174]],

         [[ 0.0211,  0.0282,  0.0009,  ...,  0.0579, -0.0752,  0.0016],
        

In [124]:
model.backbone.mtl_blocks[1].residual_block[0].weight

Parameter containing:
tensor([[[[-0.0740, -0.0166, -0.0026,  ..., -0.0646,  0.0398, -0.0464],
          [ 0.0212,  0.0403, -0.0635,  ..., -0.0483,  0.0583,  0.0641],
          [ 0.0627, -0.0651,  0.0264,  ...,  0.0188,  0.0042, -0.0241],
          ...,
          [ 0.0372,  0.0807,  0.0695,  ...,  0.0669, -0.0504, -0.0239],
          [ 0.0409, -0.0658,  0.0207,  ..., -0.0390, -0.0452, -0.0001],
          [-0.0284,  0.0345,  0.0419,  ...,  0.0402, -0.0708, -0.0144]],

         [[ 0.0271, -0.0333, -0.0447,  ...,  0.0186, -0.0489,  0.0639],
          [ 0.0666, -0.0665, -0.0644,  ...,  0.0128,  0.0039, -0.0491],
          [-0.0142,  0.0373,  0.0311,  ..., -0.0638,  0.0064,  0.0195],
          ...,
          [ 0.0204, -0.0127, -0.0747,  ..., -0.0532, -0.0158, -0.0643],
          [-0.0613,  0.0229,  0.0760,  ...,  0.0331, -0.0586, -0.0271],
          [-0.0322, -0.0743,  0.0629,  ..., -0.0697,  0.0611, -0.0154]],

         [[ 0.0220,  0.0302,  0.0029,  ...,  0.0579, -0.0753,  0.0015],
        

In [125]:
model.backbone.mtl_blocks[2].residual_block[0].weight

Parameter containing:
tensor([[[[-0.0760, -0.0185, -0.0045,  ..., -0.0666,  0.0398, -0.0463],
          [ 0.0192,  0.0383, -0.0655,  ..., -0.0483,  0.0584,  0.0642],
          [ 0.0607, -0.0671,  0.0245,  ...,  0.0189,  0.0043, -0.0241],
          ...,
          [ 0.0353,  0.0788,  0.0675,  ...,  0.0649, -0.0503, -0.0258],
          [ 0.0389, -0.0678,  0.0187,  ..., -0.0410, -0.0471, -0.0021],
          [-0.0303,  0.0325,  0.0399,  ...,  0.0383, -0.0728, -0.0163]],

         [[ 0.0251, -0.0353, -0.0466,  ...,  0.0166, -0.0488,  0.0640],
          [ 0.0646, -0.0685, -0.0664,  ...,  0.0109,  0.0040, -0.0491],
          [-0.0162,  0.0353,  0.0291,  ..., -0.0658,  0.0065,  0.0196],
          ...,
          [ 0.0184, -0.0147, -0.0767,  ..., -0.0552, -0.0178, -0.0662],
          [-0.0633,  0.0209,  0.0740,  ...,  0.0312, -0.0605, -0.0290],
          [-0.0342, -0.0763,  0.0609,  ..., -0.0716,  0.0591, -0.0174]],

         [[ 0.0211,  0.0282,  0.0009,  ...,  0.0579, -0.0752,  0.0016],
        

In [126]:
model.backbone.mtl_blocks[6].residual_block.block.conv1.weight

Parameter containing:
tensor([[[[-0.0264,  0.0124, -0.0294],
          [-0.0414,  0.0370,  0.0187],
          [-0.0356,  0.0245,  0.0311]],

         [[-0.0164,  0.0221,  0.0415],
          [ 0.0015, -0.0270, -0.0159],
          [ 0.0079,  0.0404,  0.0092]],

         [[ 0.0193,  0.0270,  0.0405],
          [ 0.0219, -0.0082, -0.0033],
          [-0.0098,  0.0075, -0.0416]],

         ...,

         [[-0.0069,  0.0146, -0.0260],
          [-0.0408, -0.0208,  0.0249],
          [-0.0322,  0.0303,  0.0215]],

         [[-0.0173, -0.0208,  0.0391],
          [-0.0306, -0.0417, -0.0416],
          [-0.0092,  0.0004, -0.0018]],

         [[-0.0071, -0.0223, -0.0065],
          [-0.0184, -0.0060,  0.0198],
          [-0.0090,  0.0293,  0.0274]]],


        [[[ 0.0199,  0.0045, -0.0202],
          [ 0.0176, -0.0193,  0.0414],
          [-0.0031,  0.0038, -0.0164]],

         [[-0.0081, -0.0184, -0.0119],
          [ 0.0331, -0.0231, -0.0117],
          [ 0.0111,  0.0233, -0.0072]],

         

In [127]:
model.backbone.mtl_blocks[7].residual_block.block.conv1.weight

Parameter containing:
tensor([[[[-2.6498e-02,  1.2349e-02, -2.9445e-02],
          [-4.1513e-02,  3.6929e-02,  1.8613e-02],
          [-3.5653e-02,  2.4430e-02,  3.1037e-02]],

         [[-1.4455e-02,  2.4065e-02,  4.3453e-02],
          [ 3.3981e-03, -2.5066e-02, -1.3982e-02],
          [ 9.8646e-03,  4.2333e-02,  1.1131e-02]],

         [[ 2.1288e-02,  2.7099e-02,  4.0583e-02],
          [ 2.3909e-02, -6.1896e-03, -1.3408e-03],
          [-7.7648e-03,  9.4680e-03, -3.9623e-02]],

         ...,

         [[-7.0012e-03,  1.4525e-02, -2.6119e-02],
          [-4.0994e-02, -2.0879e-02,  2.4815e-02],
          [-3.2301e-02,  3.0198e-02,  2.1421e-02]],

         [[-1.5335e-02, -1.8848e-02,  3.9023e-02],
          [-2.8698e-02, -4.1721e-02, -4.1683e-02],
          [-7.2550e-03,  2.3065e-03,  1.7030e-04]],

         [[-5.1677e-03, -2.2330e-02, -6.5547e-03],
          [-1.8415e-02, -6.0830e-03,  1.9763e-02],
          [-9.0489e-03,  2.9231e-02,  2.7332e-02]]],


        [[[ 1.7996e-02,  2.5151

In [128]:
model.backbone.mtl_blocks[8].residual_block.block.conv1.weight

Parameter containing:
tensor([[[[-0.0264,  0.0124, -0.0294],
          [-0.0414,  0.0370,  0.0187],
          [-0.0356,  0.0245,  0.0311]],

         [[-0.0164,  0.0221,  0.0415],
          [ 0.0015, -0.0270, -0.0159],
          [ 0.0079,  0.0404,  0.0092]],

         [[ 0.0193,  0.0270,  0.0405],
          [ 0.0219, -0.0082, -0.0033],
          [-0.0098,  0.0075, -0.0416]],

         ...,

         [[-0.0069,  0.0146, -0.0260],
          [-0.0408, -0.0208,  0.0249],
          [-0.0322,  0.0303,  0.0215]],

         [[-0.0173, -0.0208,  0.0391],
          [-0.0306, -0.0417, -0.0416],
          [-0.0092,  0.0004, -0.0018]],

         [[-0.0071, -0.0223, -0.0065],
          [-0.0184, -0.0060,  0.0198],
          [-0.0090,  0.0293,  0.0274]]],


        [[[ 0.0199,  0.0045, -0.0202],
          [ 0.0176, -0.0193,  0.0414],
          [-0.0031,  0.0038, -0.0164]],

         [[-0.0081, -0.0184, -0.0119],
          [ 0.0331, -0.0231, -0.0117],
          [ 0.0111,  0.0233, -0.0072]],

         

# For Visualize

In [131]:
from graphviz import Digraph
from torch.autograd import Variable

In [135]:
def make_dot(var, params=None):
    if params is not None:
        assert isinstance(params.values()[0], Variable)
        param_map = {id(v): k for k, v in params.items()}

    node_attr = dict(style="filled", shape="box", align="left", fontsize="12", ranksep="0.1", height="0.2")
    dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))
    seen = set()

    def size_to_str(size):
        return "(" + (", ").join(["%d" % v for v in size]) + ")"

    def add_nodes(var):
        if var not in seen:
            if torch.is_tensor(var):
                dot.node(str(id(var)), size_to_str(var.size()), fillcolor="orange")
                dot.edge(str(id(var.grad_fn)), str(id(var)))
                var = var.grad_fn
            if hasattr(var, "variable"):
                u = var.variable
                name = param_map[id(u)] if params is not None else ""
                node_name = "%s\n %s" % (name, size_to_str(u.size()))
#                 print(node_name)
                
                dot.node(str(id(var)), node_name, fillcolor="lightblue")
            else:
                print(type(var).__name__)
                
                dot.node(str(id(var)), str(type(var).__name__))
            seen.add(var)
            if hasattr(var, "next_functions"):
                for u in var.next_functions:
                    if u[0] is not None:
                        dot.edge(str(id(u[0])), str(id(var)))
                        add_nodes(u[0])
            if hasattr(var, "saved_tensors"):
                for t in var.saved_tensors:
                    dot.edge(str(id(t)), str(id(var)))
                    add_nodes(t)
    
    for v in var:
        add_nodes(var[v])
    return dot

In [136]:
inputs = torch.randn(1, 3, 224, 224)
y = model(inputs.cuda())
g = make_dot(y)
g.view()

AddBackward0
AddBackward0
AddBackward0
AddBackward0
CudnnConvolutionBackward
FusedDropoutBackward
ReluBackward1
AddBackward0
CudnnConvolutionBackward
FusedDropoutBackward
ReluBackward1
AddBackward0
CudnnConvolutionBackward
ReluBackward0
AddBackward0
ReluBackward0
AddBackward0
ReluBackward0
AddBackward0
CudnnBatchNormBackward
CudnnConvolutionBackward
ReluBackward0
AddBackward0
ReluBackward0
AddBackward0
ReluBackward0
AddBackward0
ReluBackward0
AddBackward0
ReluBackward0
AddBackward0
ReluBackward0
AddBackward0
CudnnBatchNormBackward
CudnnConvolutionBackward
ReluBackward0
AddBackward0
ReluBackward0
AddBackward0
ReluBackward0
AddBackward0
ReluBackward0
AddBackward0
CudnnBatchNormBackward
CudnnConvolutionBackward
ReluBackward0
AddBackward0
ReluBackward0
AddBackward0
ReluBackward0
AddBackward0
MaxPool2DWithIndicesBackward
ReluBackward1
CudnnBatchNormBackward
CudnnConvolutionBackward
CudnnBatchNormBackward
CudnnConvolutionBackward
ReluBackward1
CudnnBatchNormBackward
CudnnConvolutionBackward


'Digraph.gv.pdf'

In [9]:
import torch.onnx

In [11]:
PATH = "models/mtlmodel_30_manual.onnx"

x = torch.rand(1,3,224,224)
torch.onnx.export(model,x.cuda(),PATH)